# Solar Power

## Business Understanding

### Stakeholder:
- 
-
-

### True Business Problem:
With a recent push for cleaner renewable energy (company name) is seeing an increase in solar panel request. (company name) is looking the potential of solar power and how it can meet the demands of the current power consumption. 

# Data Understanding

In [3]:
# imports
import pandas as pd
import numpy as np
import requests 

from datetime import datetime as dt
from secret import keys
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

!pip install sktime
!pip install pmdarima
!pip install pystan
!pip install fbprophet

  Using cached sktime-0.13.0-py3-none-any.whl (6.8 MB)
  Using cached scikit_learn-1.1.1-cp38-cp38-win_amd64.whl (7.3 MB)
  Using cached Deprecated-1.2.13-py2.py3-none-any.whl (9.6 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 0.17.0
    Uninstalling joblib-0.17.0:
      Successfully uninstalled joblib-0.17.0

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\FFF\\anaconda3\\envs\\learn-env\\Lib\\site-packages\\~klearn\\metrics\\cluster\\_expected_mutual_info_fast.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2


In [9]:
!ls ../data

df2.csv


In [15]:
df = pd.read_csv('../data/df2.csv')
df.head()

,period,NG,OIL,SUN
0,2021-07-01 00:00:00,20540,-1,177
1,2021-07-01 01:00:00,19923,0,32
2,2021-07-01 02:00:00,18983,0,0
3,2021-07-01 03:00:00,17476,-1,0
4,2021-07-01 04:00:00,15457,0,0


0        177
1         32
2          0
3          0
4          0
        ... 
9427    1099
9428    1006
9429     798
9430     765
9431     484
Name: SUN, Length: 9432, dtype: int64